# Linear Regression in Python

There are many packages that implement linear regression in Python. As detailed in our last reading, however, depending on whether they are intended for use in prediction or inference, the way these packages operate can vary substantially. 

In this reading, we will look at how linear regression has been implemented in two major packages — [statsmodels](http://statsmodels.org) and [scikit-learn](http://scikit-learn.org). Both of these packages can fit a wide range of linear models, but the way they are organized and the results they report reflect the different audiences for whom they were designed. 

*Broadly speaking*, statsmodels is a library written by statisticians for statisticians, biostatisticians, social scientists, and natural scientists. It can do prediction, but its focus is *inference*, and as we will see that is reflected throughout the package. 

scikit-learn, by contrast, was written by and for computer scientists interested in machine learning. Its focus is on prediction, and while it includes a far more diverse collection of machine learning models than statsmodels, it does not include all the features someone doing inference might expect for evaluating model performance or doing things like calculating different types of standard errors.

## Regression in statsmodels

Because it is the more feature-rich library when it comes to regression, we will start our exploration of linear regression in Python with statsmodels. If you have any interest in inference, are coming from a programming language like R or Stata, and/or have a background in statistics, social science, or the natural sciences, then statsmodels is the package that will feel most familiar and useful. 

While our focus will be on linear regression, the statsmodels package includes a wide range of tools for inference and modeling, from simple models like linear and logistic regression to generalized linear models (GLMs), non-parametric regression, robust linear models, time series models, survival analysis, multiple imputation, generalized additive models (GAMs), and more. (Curious if it includes that *one* model that's near and dear to your heart? [Feel free to go check and come back](https://www.statsmodels.org/stable/user-guide.html)).

Moreover, it provides by far the easiest interface for moving from a pandas DataFrame to a regression in the Python ecosystem. To illustrate, let's fit a quick regression looking at countries' under-5 mortality rates as a function of GDP per capita and Work Bank ratings of each countries' public sector in terms of transparency, accountability, and levels of corruption:

In [25]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

pd.set_option("mode.copy_on_write", True)

# Load data on infant mortality, gdp per capita, and
# World Bank CPIA public sector transparency, accountability,
# and corruption in the public sector scores
# (1 = low transparency and accountability, 6 = high transparency and accountability).

wdi = pd.read_csv("data/wdi_corruption.csv")

# Check one observation to get a feel for things.
wdi.sample().T

,38
country_name,Mali
gdp_per_capita_ppp,1922.434
CPIA_public_sector_rating,3.0
mortality_rate_under5_per_1000,113.7
"Mortality rate, under-5, female (per 1,000 live births)",107.8
"Mortality rate, under-5, male (per 1,000 live births)",119.0
"Population, total",17438778.0
region,Sub-Saharan Africa


In [ ]:
# Fit model
corruption_model = smf.ols(
    "mortality_rate_under5_per_1000 ~ np.log(gdp_per_capita_ppp) + CPIA_public_sector_rating + region",
    data=wdi,
).fit()

# Get regression result
corruption_model.summary()

(why `smf.ols`? Recall that one name for linear regression is "Ordinary Least Squares," which is often shortened to "OLS.")

Not bad, huh? Not only was our regression a one-liner, but the `.summary()` method has provided us with all our coefficients with labels, standard errors, t-statistics, p-values, 95% confidence intervals, and a range of additional statistics about the regression. 

We were also able to apply come manipulations right in the regression specification — note the use of `np.log()` to apply the log function to `gdp_per_capita_ppp` without having to create a new variable in our DataFrame.

Finally (if this doesn't mean much to you, don't worry about it), statsmodels recognized that `region` contained strings rather than numbers, so it dynamically created a set of indicator variables (i.e., it created one-hot encodings).

If you come from R or Stata, none of that is likely to seem notable to you, but as we'll discuss in the next section (and as we'll see when we get to `scikit-learn`), those are conveniences that should not be taken for granted.

## Moving from Pandas to Numpy

Tools like the `ols` function from `statsmodels.formula.api` we used above can make your life as a data scientist *much* easier, but they also mask some of the complexity that lies between passing the function a pandas DataFrame and the actual regression taking place. 

Why? At the level of the underlying math, a linear regression is fit using matrix calculations. Indeed, if you imagine putting your outcome variable in vector called $y$ and your explanatory variables in a matrix $X$ where each column one of your explanatory variables and each row is an observation, then the way we actually get the vector of coefficients $\beta$ is with the following matrix formula:

$$\beta = (X^TX)^{-1} X^Ty$$

Indeed, matrix math (also known as matrix algebra) underlies a huge proportion of statistics and machine learning tools, which is one of the reasons that a linear algebra course (the part of math focused on the study of matrix math) is often recommended for data science students. (Don't worry — you don't need to know linear algebra to start as a data scientist, or to understand that formula above; I'm just telling you this to motivate what comes next).

But you can't always do matrix math on a pandas DataFrame. pandas DataFrames are great because of their flexibility, but you can't multiply to pandas DataFrames that contain some integer columns, some floating point columns, and some string columns. Moreover, you also can't do matrix calculations with missing values — the inverse of a matrix with missing values is not a defined concept. 

So how was `smf.ols()` able to fit that regression? [patsy](https://patsy.readthedocs.io/en/latest/overview.html). patsy is a package that takes as input a pandas DataFrame and a formula (like the `"mortality_rate_under5_per_1000 ~ np.log(gdp_per_capita_ppp) + CPIA_public_sector_rating + region"` formula we used above), and returns a numpy array of floating point numbers without missing data (and with labels!) on which matrix algebra can be performed. And all `smf.ols()` is doing is calling patsy, giving it your DataFrame and formula, the taking the numpy arrays it returns and passing them to its more traditional OLS function, `OLS()` from `statsmodels.api`.


### Why do I care?

"OK," I hear you yell, "but if statsmodels does this for me what do I care how it does it?"

You need to care because outside a handful of functions for which statsmodels offers this convenience (you can find the [full list here](https://www.statsmodels.org/stable/api.html#statsmodels-formula-api)), **nearly every modeling tool you encounter in the Python ecosystem will expect numpy arrays of floating point numbers as inputs.** And that means that it's important to (a) understand why they're doing this, and (b) be comfortable using `patsy` directly to move from pandas DataFrames to numpy arrays yourself.

## patsy Formula Syntax

To move between pandas and numpy, we will use the `patsy.dmatrices()` function. It takes a pandas DataFrame and a formula string and returns a tuple containing an outcome vector and a matrix of explanatory variables.

To illustrate how patsy works, let's create a quite small toy dataset just to help us visualize what is going on. 

In [56]:
toy_df = pd.DataFrame(
    {
        "y": [10, 11, 9, 8, 12],
        "a": [1, 2, 3, 1, 3],
        "b": [3.14, np.nan, 2.5, 0, 3],
        "names": ["Nick", "Lyra", "Nick", "Lyra", "Lyra"],
    }
)
toy_df

,y,a,b,names
0,10,1,3.14,Nick
1,11,2,NaN,Lyra
2,9,3,2.50,Nick
3,8,1,0.00,Lyra
4,12,3,3.00,Lyra


Now let's suppose we want to regress `y` on `a` and `b`. Let's use `dmatrices` to create our matrices. Note that `patsy.dmatrices` returns two objects — our outcome vector and our matrix of explanatory variables, so if you assign the result to a single variable you will get a `tuple` with the two items, or you can assign the result to two variables (separated by a comma) and the results will be distributed to those two variables:

In [55]:
import patsy

y, X = patsy.dmatrices("y ~ a + b", data=toy_df)

Let's start by looking at our X matrix:

In [54]:
X

DesignMatrix with shape (4, 2)
  a     b
  1  3.14
  3  2.50
  1  0.00
  3  3.00
  Terms:
    'a' (column 0)
    'b' (column 1)

As you can see, there are a few differences between our pandas DataFrame and this `DesignMatrix` (these are basically numpy arrays with some extra metadata, like labels. You can use them with any function that expects numpy arrays).

- The DesignMatrix `X` includes a column of 1s labelled as `Intercept`. You may have never thought about this before, but the intercept in a regression doesn't appear by magic — it's actually a coefficient created by adding a column that has the same value for every observation (1 makes interpretation easiest). By default, `patsy` will always add an intercept, but if you would prefer it did not, just add `- 1` or `+ 0` to the formula.
- Our pandas DataFrame had 5 rows, but `X` only has 4. Why? Well, the second row of our DataFrame included a missing value (`np.nan`) in the `b` column. As noted above, we can't do matrix algebra with missing values, so any row that includes a missing value *in one of the variables in the formula* is automatically dropped. And yes, it drops that row from `y` too.

In [51]:
y

DesignMatrix with shape (4, 1)
   y
  10
   9
   8
  12
  Terms:
    'y' (column 0)

### Categorical Variables

Great, but this is just the start of things. Now let's add `names` — a string variable — to the model.

In [57]:
y, X = patsy.dmatrices("y ~ a + b + names", data=toy_df)
X

DesignMatrix with shape (4, 4)
  Intercept  names[T.Nick]  a     b
          1              1  1  3.14
          1              1  3  2.50
          1              0  1  0.00
          1              0  3  3.00
  Terms:
    'Intercept' (column 0)
    'names' (column 1)
    'a' (column 2)
    'b' (column 3)

Now our `X` matrix has a new column called `names[T.Nick]` that contains 0s and 1s. Basically, if the variable takes on the value after `T.` (here, `Nick`), then that variable will be 1; otherwise it will be a zero. If our variable `c` took on more than two values (say, `"Lyra"`, `"Nick",` `"Trillian"`), `X` would have also had a variable called `names[T.Trillian]`.

By default, patsy will always omit one category (the "omitted category" or "reference category"). As a result, for a variable that takes on `N` unique values, patsy will always add `N-1` variables when doing one-hot encoding.

patsy will do this automatically when it encounters a `Categorical` or `object` `dtype` column, but you can also use `C()` to force it to treat any variable as categorical. For example, if I put `C(a)` in the formula, it will treat `a` as a categorical variable and apply one-hot encoding to all the unique values of the variable:

In [60]:
y, X = patsy.dmatrices("y ~ C(a) + b", data=toy_df)
X

DesignMatrix with shape (4, 4)
  Intercept  C(a)[T.2]  C(a)[T.3]     b
          1          0          0  3.14
          1          0          1  2.50
          1          0          0  0.00
          1          0          1  3.00
  Terms:
    'Intercept' (column 0)
    'C(a)' (columns 1:3)
    'b' (column 3)

Arguments can also be passed to `C()` to modify patsy's one-hot encoding behavior. For example, if you pass `C(names, Treatment('Nick'))`, `"Nick"` will become the omitted category (instead of the default, which is the first value alphabetically):

In [62]:
y, X = patsy.dmatrices("y ~ a + b + C(names, Treatment('Nick'))", data=toy_df)
X

DesignMatrix with shape (4, 4)
  Intercept  C(names, Treatment('Nick'))[T.Lyra]  a     b
          1                                    0  1  3.14
          1                                    0  3  2.50
          1                                    1  1  0.00
          1                                    1  3  3.00
  Terms:
    'Intercept' (column 0)
    "C(names, Treatment('Nick'))" (column 1)
    'a' (column 2)
    'b' (column 3)

### Interaction Terms

Patsy can also create interaction terms on the fly. For example, if you wished to add the multiplicative interaction of `a` and `b`, you could either put `a*b` in place of `a + b` to have patsy insert `a`, `b`, and `a*b` to the model:

In [58]:
y, X = patsy.dmatrices("y ~ a * b + names", data=toy_df)
X

DesignMatrix with shape (4, 5)
  Intercept  names[T.Nick]  a     b   a:b
          1              1  1  3.14  3.14
          1              1  3  2.50  7.50
          1              0  1  0.00  0.00
          1              0  3  3.00  9.00
  Terms:
    'Intercept' (column 0)
    'names' (column 1)
    'a' (column 2)
    'b' (column 3)
    'a:b' (column 4)

Or you can leave in `a + b` and add `a:b` to just add the multiplicative interaction term:

In [59]:
y, X = patsy.dmatrices("y ~ a * b + names", data=toy_df)
X

DesignMatrix with shape (4, 5)
  Intercept  names[T.Nick]  a     b   a:b
          1              1  1  3.14  3.14
          1              1  3  2.50  7.50
          1              0  1  0.00  0.00
          1              0  3  3.00  9.00
  Terms:
    'Intercept' (column 0)
    'names' (column 1)
    'a' (column 2)
    'b' (column 3)
    'a:b' (column 4)

### Isn't this formula syntax the same as R?

Pretty close! Here's a [list of all known differences](https://patsy.readthedocs.io/en/latest/R-comparison.html), but as you'll see, there aren't too many. 